In [ ]:
import numpy as np
#import xml.etree.ElementTree as ET
from lxml import objectify

In [2]:
# I don't want to have to redo all the smarts definitions
with open("uff-copy.xml", "r") as f:
    data=f.readlines()
xml_str = "".join(data)
root = objectify.fromstring(xml_str)

In [3]:
uff_dict = {}
for a in root.AtomTypes[0].Type:
    #print(a.attrib)
    try:
        defn = a.attrib['def']
    except KeyError:
        defn = None
    try:
        desc = a.attrib['desc']
    except KeyError:
        desc = None
    try:
        overrides = a.attrib['overrides']
    except KeyError:
        overrides = None
    a_dict = {
        "def" : defn,
        "desc" : desc,
        "overrides" : overrides
    }
    uff_dict[a.attrib["name"]] = a_dict

In [5]:
# angstroem, deg, kcal/mol, charge
# type, element, bond, angle, distance, energy, scale, charge
d_uff = np.dtype([
    ("type", np.unicode_, 8), 
    ("element", np.unicode_, 2), 
    ("bond", "d"),
    ("angle", "d"),
    ("distance", "d"),
    ("energy", "d"),
    ("scale", "d"),
    ("charge", "d"),
])
uff_data = np.loadtxt("uff.txt", dtype=d_uff)
#print(uff_data[0]["type"])

In [6]:
mass_dict = {}
with open("masses.txt") as f:
    for line in f.readlines():
        elem, mass = line.split()
        mass_dict[elem] = mass
#print(mass_dict)

In [12]:
with open("uff.xml", "w") as f:
    f.write('<ForceField version="0.0.1" name="UFF">\n\t<FFMetaData>\n')
    f.write('\t\t<Units energy="kcal/mol" distance="angstrom" mass="amu" charge="e"/>\n')
    f.write('\t</FFMetaData>\n\t<AtomTypes expression="4*epsilon*((sigma/r)**12 - (sigma/r)**6)">\n')
    f.write('\t\t<ParametersUnitDef parameter="sigma" unit="angstrom"/>\n')
    f.write('\t\t<ParametersUnitDef parameter="epsilon" unit="kcal/mol"/>\n')
    for atom in uff_data:
        a_str = ""
        for param in ["def", "overrides", "desc"]:
            try:
                val = uff_dict[atom["type"]][param]
            except KeyError:
                val = None
            if val != None:
                a_str += f' {param} = "{val}"'
        try:
            f.write(
                '\t\t<AtomType name="{}" element="{}" mass="{}"{}/>\n'.format(
                    atom["type"], atom["element"], mass_dict[atom["element"]], a_str
                )
            )
            f.write(
                '\t\t\t<Parameters>\n\t\t\t\t<Parameter name="sigma" value="{}">\n'.format(
                atom["distance"]
                )
            )
            f.write(
                '\t\t\t\t<Parameter name="epsilon" value="{}">\n\t\t\t</Parameters>\n'.format(
                atom["energy"]
                )
            )
            f.write('\t\t</AtomType>\n')
        except KeyError:
            pass # if we don't have the mass, skip it for now
    f.write('\t</AtomTypes>\n')
    f.write('\t<HarmonicBondForce expresion="1/2*k*(r-length)**2">\n')
    f.write('\t<ParametersUnitDef parameter="length" unit="angstrom"/>\n')
    f.write('\t<ParametersUnitDef parameter="k" unit="(kcal/mol)/angstrom**2"/>\n')
    for i,atom1 in enumerate(uff_data):
        for atom2 in uff_data[i:]:
            try:
                mass_dict[atom1["element"]]
                mass_dict[atom2["element"]]
            except KeyError:
                continue
            # so in the rappe paper r_ij is supposed to be 
            # r_ij = r_i + r_j +r_BO + r_EN 
            # but to simplify things, I'm going to try just 
            # r_ij = r_i + r_j + 0.026
            # 0.026 is a correction so the c-c bond comes out right.
            length = atom1["bond"] + atom2["bond"] + 0.026 #angstroem
            
            # k_ij = 664.12 (Z*_i x Z*_j)/(r_ij^3) 
            energy = 664.12 * atom1["charge"] * atom2["charge"]/ length**3
            
            f.write(
                '\t\t<Bond name1="{}" name2="{}" length="{:.3f}" k="{:.3f}"/>\n'.format(
                    atom1["type"], atom2["type"], length, energy
                )
            )

    f.write('\t</HarmonicBondForce>\n')
    f.write('</ForceField>')